# Metadata Explorer

A notebook to explore the metadata so we know where to look for things!

### Import some libraries

In [1]:
import os
import sys

import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
from matplotlib.colors import ListedColormap
import pandas as pd
import seaborn as sns
import json

from itertools import cycle

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.inspection import permutation_importance

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error

from scipy.stats import linregress


# there is a FutureWarning in sklearn StandardScalar which is really annoying. This ignores it.
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

try:
  import google.colab
  IN_COLAB = True
  !pip install adjustText
  from google.colab import drive
  drive.mount('/content/drive')
  datadir = '/content/drive/MyDrive/Projects/CF/Adelaide/CF_Data_Analysis'
except ImportError:
  IN_COLAB = False
  datadir = '..'

from adjustText import adjust_text

import cf_analysis_lib

### Read the data

In [2]:
sequence_type = 'MGI_minion'
metadata = cf_analysis_lib.read_metadata(datadir, sequence_type)
metadata.head(5)

In [3]:
sorted(list(metadata.columns))

In [5]:
antibiotics = ['1 Cephalexin_PO', '1 Flucloaxcillin_PO', '1 Itraconazole (Lozenoc)_PO', '1 Sulfamethoxazole_trimethoprim (Bactrim)_PO', '2 Amikacin_INH', '2 Amoxicillin & Potassium clavulanate (Aug Duo)_PO', '2 Amphotericin B (Ambisome)_INH', '2 Azithromycin_PO', '2 Ceftazidime_INH', '2 Ciprofloxacin_PO', '2 Clarithromycin_PO', '2 Clofazimine PO', '2 Colistin_IHN', '2 prednisolone_PO', '2 tobramycin_INH', '3 Azithromycin_IV', '3 Aztreonam_IV', '3 Cefopime_IV', '3 Ceftazidime_IV', '3 Imipenem', '3 Ivacaftor (Kalydeco)', '3 Meropenem_IV', '3 Methylpredinosolone_IV', '3 Omalizumab_SC', '3 piperacillin sodium, tazobactam sodium (Tazocin)_IV', '3 tobramycin_IV', '4 Amikacin_IV', '4 Cefoxitin_IV', '4 Colistin_IV']

In [6]:
antibiotics

In [7]:
combinations = {
'Amikacin' : ['2 Amikacin_INH', '4 Amikacin_IV'],
'Azithromycin' : ['2 Azithromycin_PO', '3 Azithromycin_IV'],
'Tobramycin' : ['2 tobramycin_INH', '3 tobramycin_IV'],
'Colistin' : ['2 Colistin_IHN', '4 Colistin_IV'],
'Ceftazidime' : ['2 Ceftazidime_INH', '3 Ceftazidime_IV'],
'Prednisolone' : ['2 prednisolone_PO', '3 Methylpredinosolone_IV']
}

In [12]:
metadata[(metadata['2 Amikacin_INH'] == 1) | (metadata['4 Amikacin_IV'] == 1)]

In [13]:
tmpdf = pd.DataFrame()
for new_col, col_array in combinations.items():
    tmpdf[new_col] = metadata[col_array].any(axis=1).astype(int)
tmpdf

In [14]:
tmpdf.sum()